In [1]:
import numpy as np

### Broadcast

**The Broadcasting Rule**

>In order to broadcast, the size of the trailing axes for both arrays in an operation must either be the same size or one of them must be one.


``` ndarray(n,m,l) * ndarray(m,l) ``` OR ``` ndarray(n,m,l) * ndarray(1,1) ```

In [2]:
a = np.random.random([20,12,5,7,2,8,4])

In [3]:
%%prun
a * 8

         3 function calls in 0.003 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.002    0.002 <string>:1(<module>)
        1    0.000    0.000    0.003    0.003 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [4]:
%%prun
eight = np.full(a.shape,8)
a * eight

         10 function calls in 0.004 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.003    0.003    0.004    0.004 <string>:1(<module>)
        1    0.001    0.001    0.001    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.004    0.004 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.empty}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.array}
        1    0.000    0.000    0.001    0.001 numeric.py:288(full)
        1    0.000    0.000    0.001    0.001 <__array_function__ internals>:2(copyto)
        1    0.000    0.000    0.000    0.000 _asarray.py:23(asarray)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 multiarray.py:1054(copyto)

In [5]:
x = np.arange(5)
x[:,np.newaxis] + x[np.newaxis,:]

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 5],
       [2, 3, 4, 5, 6],
       [3, 4, 5, 6, 7],
       [4, 5, 6, 7, 8]])

#### Vector quantization

![VQ](https://numpy.org/doc/stable/_images/theory.broadcast_5.png)

In [6]:
bp = np.array([[205],[112]])
fl = np.array([[195],[148]])
mr = np.array([[175],[68]])
fg = np.array([[155],[50]])

In [7]:
new = np.array([[187],[126]])

In [8]:
athletes = np.hstack((bp,fl,mr,fg))
athletes

array([[205, 195, 175, 155],
       [112, 148,  68,  50]])

**Euclidean distance**
$$
\sqrt{\sum_{i = 1}^{n}(p_i - q_i)^2} 
$$
**Finding closest** $ {Athletes}_c\ , c = \arg \min \sqrt{\sum_{i = 1}^{n}(p_i - q_i)^2}  $

In [9]:
arg = np.argmin(
    np.sqrt(
        np.sum(
            np.power( (athletes - new), 2)
            ,axis=0)
    ) 
)

In [10]:
athletes[:,arg] ##backetball player

array([205, 112])

### Structured arrays

In [11]:
struct = np.array([('Bob',42), ('Alice',39)],
                  dtype = [('name','U5'),('age','i1')])

In [12]:
struct

array([('Bob', 42), ('Alice', 39)], dtype=[('name', '<U5'), ('age', 'i1')])

In [13]:
struct['name']

array(['Bob', 'Alice'], dtype='<U5')

In [14]:
_dtype = np.dtype([('name','U5'),('age',np.uint8)])
struct = np.array([('Nick',15),('Sofia',17),('Helen',14)],dtype=_dtype)

In [15]:
struct

array([('Nick', 15), ('Sofia', 17), ('Helen', 14)],
      dtype=[('name', '<U5'), ('age', 'u1')])

In [16]:
struct['age']

array([15, 17, 14], dtype=uint8)

### Indexing

In [17]:
a = np.arange(12).reshape(3,4)
i = np.array([[1,2],[0,1]])
j = i.view()

In [18]:
a[i]

array([[[ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[ 0,  1,  2,  3],
        [ 4,  5,  6,  7]]])

In [19]:
a[i,j]

array([[ 5, 10],
       [ 0,  5]])

In [20]:
a[i][0]

array([[ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [21]:
a[i].shape

(2, 2, 4)

#### Note:
`a[i,j] is faster than a[i][j]`

In [22]:
%%prun
a = np.random.rand(40,12,15,12,9,6,10)

         4 function calls in 0.424 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.424    0.424    0.424    0.424 {method 'rand' of 'numpy.random.mtrand.RandomState' objects}
        1    0.000    0.000    0.424    0.424 {built-in method builtins.exec}
        1    0.000    0.000    0.424    0.424 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [23]:
%%time
a[2,7,3,1,4,5,0]

Wall time: 0 ns


0.8745653810892984

In [24]:
%%time
a[2][7][3][1][4][5][0]

Wall time: 0 ns


0.8745653810892984

In [25]:
a[2,7,3,1,4,Ellipsis,0]

array([0.75887547, 0.25342112, 0.72226307, 0.04232523, 0.50731026,
       0.87456538])

### Custom array containers and classes

#### Containers
> Array containers do not inherit from the NumPy array classes. Containers **contain** a magic method `__array__`  that allows you to turn into a numpy array using the `array()` or `asarray()` functions.

> Note that `__array__` must produce an array.

In [26]:
class CustomContainer:
    def __init__(self, size: tuple, val: float):
        self._val = val
        self._size = size
        self._array = None
    def __array__(self, dtype=None):
        ### producing an array
        ### with unpacking the tuple
        self._array = np.sin(np.eye(*self._size) * self._val)
        return self._array

In [27]:
container = CustomContainer((4,4),9)
np.array(container, dtype=float)

array([[0.41211849, 0.        , 0.        , 0.        ],
       [0.        , 0.41211849, 0.        , 0.        ],
       [0.        , 0.        , 0.41211849, 0.        ],
       [0.        , 0.        , 0.        , 0.41211849]])

In [28]:
try:
    print(container * 2)
except TypeError:
    print(np.asarray(container) * 2)

[[0.82423697 0.         0.         0.        ]
 [0.         0.82423697 0.         0.        ]
 [0.         0.         0.82423697 0.        ]
 [0.         0.         0.         0.82423697]]


In [29]:
class MulContainer(CustomContainer):
    def __init__(self, size: tuple, val: float):
        super().__init__(size, val)
        self.__array__(self)
    def __mul__(self, b):
        return self._array * b

In [30]:
mc = MulContainer((3,3),90)
mc * 2

array([[1.78799333, 0.        , 0.        ],
       [0.        , 1.78799333, 0.        ],
       [0.        , 0.        , 1.78799333]])

#### Subclasses
There are 3 types of nd.array subclass creation:
* View
* Template
* Explicit

In [31]:
class FromViewSClass(np.ndarray): pass

arr = np.eye(4,4)*9
## view method
## subclass is `type` parameter
arr = arr.view(FromViewSClass)
type(arr)

__main__.FromViewSClass

Array container is not subclass and it haven't the `.sum()` method
``` python
mc.sum() ##error
```

In [32]:
arr.sum()

FromViewSClass(36.)

In [33]:
if arr[:] is not arr:
    ## instance of FromViewSClass
    ## with copy of `template`
    print(type(arr[:]))

<class '__main__.FromViewSClass'>


In [34]:
import numpy as np

In [35]:
class ParamArray(np.ndarray): ## subclass of ndarray
    def __new__(cls, shape, value=1, gain=None, cost=None):
        ## __init__ not so flexible
        ## Calling CustonClass(custom_arg)
        ## Raise the exception
        ## is an invalid keyword argument for ndarray()
        instance = super().__new__(cls, shape)
        ## instead of instance.data
        instance[...] = np.sin(np.eye(*shape) * value)
        instance.cost = cost
        instance.gain = gain
        return instance
    def __array_finalize__(self, obj):
        ## from explicit instance creation
        if obj is None: return
        ## in other cases
        self[...] = np.sin(np.eye(*obj.shape) * 1)
        self.cost = getattr(obj, 'cost', None)
        self.gain = getattr(obj, 'gain', None)
    
    def tostring(self, **unused_kwargs):
        print('custom method')
        ## **unused_args caution for
        ## future numpy args update 

In [36]:
subclass = ParamArray((5,3),3, gain = -.1425)
subclass * subclass.gain

ParamArray([[-0.0201096, -0.       , -0.       ],
            [-0.       , -0.0201096, -0.       ],
            [-0.       , -0.       , -0.0201096],
            [-0.       , -0.       , -0.       ],
            [-0.       , -0.       , -0.       ]])

In [37]:
subclass = arr.view(ParamArray)
subclass

ParamArray([[0.84147098, 0.        , 0.        , 0.        ],
            [0.        , 0.84147098, 0.        , 0.        ],
            [0.        , 0.        , 0.84147098, 0.        ],
            [0.        , 0.        , 0.        , 0.84147098]])

In [38]:
subclass.tostring()

custom method


### Creation and manipulation

In [39]:
a = np.zeros([5,3])
a

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [40]:
b = np.identity(5)
b

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [41]:
c = np.full([4,3], 7); c

array([[7, 7, 7],
       [7, 7, 7],
       [7, 7, 7],
       [7, 7, 7]])

In [42]:
(b @ a @ c.T).reshape(2,-1)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [43]:
c.reshape(1,4*3)

array([[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]])

In [44]:
m = np.matrix([
    [1,2,3],
    [7,5,9],
    [12,-1,0]
])

In [45]:
np.rollaxis(m,1)

matrix([[ 1,  7, 12],
        [ 2,  5, -1],
        [ 3,  9,  0]])

In [46]:
np.concatenate((m,c.reshape(3,-1)),axis=1)

matrix([[ 1,  2,  3,  7,  7,  7,  7],
        [ 7,  5,  9,  7,  7,  7,  7],
        [12, -1,  0,  7,  7,  7,  7]])

In [47]:
np.split(np.concatenate((m,c.reshape(3,-1)),axis=1),7,axis=1)

[matrix([[ 1],
         [ 7],
         [12]]),
 matrix([[ 2],
         [ 5],
         [-1]]),
 matrix([[3],
         [9],
         [0]]),
 matrix([[7],
         [7],
         [7]]),
 matrix([[7],
         [7],
         [7]]),
 matrix([[7],
         [7],
         [7]]),
 matrix([[7],
         [7],
         [7]])]

In [48]:
np.unique(m, return_counts=True)

(matrix([[-1,  0,  1,  2,  3,  5,  7,  9, 12]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64))

In [49]:
np.flip(m,axis=[1,0])

matrix([[ 0, -1, 12],
        [ 9,  5,  7],
        [ 3,  2,  1]])

In [50]:
np.ravel(m)

array([ 1,  2,  3,  7,  5,  9, 12, -1,  0])

In [51]:
np.insert(m,obj=[0,1,2],values=[-7,-7,-7],axis=1)

matrix([[-7,  1, -7,  2, -7,  3],
        [-7,  7, -7,  5, -7,  9],
        [-7, 12, -7, -1, -7,  0]])

### Copy

In [52]:
n = m[...] ## simple copy
k = m.view() ## shallow copy
l = m.copy() ## deep copy

In [53]:
n.fill(-1) ## no copy since ndarray is mutable
## any manipulation with array change the original data
m ## 1 object, 1 data

matrix([[-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1]])

In [54]:
k.fill(0) ## create view of object
m ## 2 objects, 1 data

matrix([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [55]:
l.fill(1) ## copy data to new object
m ## 2 obj, 2 data

matrix([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

### Strings and datetime

In [56]:
np.char.count('aff0vs','f')

array(2)

In [57]:
np.char.find('abc','b')

array(1)